# Pre-processing pipeline for spikeglx sessions, zebra finch / starling
- For every run in the session:
 - Load the recordings
 - Extract wav chan with micrhopohone and make a wav chan with the nidq syn signal
 - Get the sync events for the nidq sync channel
 
 - Do bout detection
 
In another notebook, bout detection is curated
- Left to decide where to:
    - Sort spikes
    - Sync the spikes/lfp/nidq
    - make and plot 'bout rasters'

In [5]:
%matplotlib inline
import os
import glob
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal
import traceback
import warnings

from matplotlib import pyplot as plt
from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)


In [6]:
from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util import sglxutil as sglu
from ceciestunepipe.util import rigutil as ru
from ceciestunepipe.util import wavutil as wu
from ceciestunepipe.util import syncutil as su
from ceciestunepipe.util import stimutil as st
from ceciestunepipe.util.sound import boutsearch as bs
from ceciestunepipe.pipeline import searchbout as sb

from ceciestunepipe.util.spikeextractors import preprocess as pre
from ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor import readSGLX as rsgl
from ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor import spikeglxrecordingextractor as sglex

In [7]:
import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.toolkit as st
import spikeinterface.sorters as ss
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw
logger.info('all modules loaded')

2022-04-19 14:07:08,730 root         INFO     all modules loaded
2022-04-19 14:07:08,730 root         INFO     all modules loaded


## Session parameters and raw files

#### list all the sessions for this bird

In [8]:
bird = 's_b1238_22'
all_bird_sess = et.list_sessions(bird)
logger.info('all sessions for bird are {}'.format(all_bird_sess))

2022-04-19 14:07:14,279 root         INFO     all sessions for bird are ['2022-01-07', '2022-01-08', '2022-01-09', '2022-01-10', '2022-01-11', '2022-01-12', '2022-01-13', '2022-01-14', '2022-01-15', '2022-01-16', '2022-01-17', '2022-01-18', '2022-01-19', '2022-01-20', '2022-01-21', '2022-01-22', '2022-01-23', '2022-01-24', '2022-01-25', '2022-01-26', '2022-01-27', '2022-01-28', '2022-01-29', '2022-01-30', '2022-01-31', '2022-02-01', '2022-02-02', '2022-02-03', '2022-02-04', '2022-02-05', '2022-02-06', '2022-02-07', '2022-02-08', '2022-02-09', '2022-02-10', '2022-02-11', '2022-02-12', '2022-02-13', '2022-02-14', '2022-02-15', '2022-02-16', '2022-02-17', '2022-02-18', '2022-02-19', '2022-02-20', '2022-02-21', '2022-02-22', '2022-02-23', '2022-02-24', '2022-02-25', '2022-02-26', '2022-02-27', '2022-02-28', '2022-03-01', '2022-03-02', '2022-03-03', '2022-03-04', '2022-03-05', '2022-03-06', '2022-03-07', '2022-03-08', '2022-03-09', '2022-03-10', '2022-03-11', '2022-03-12', '2022-03-13', '20

### set up bird and sessions parameters
this will define:
- locations of files (for the bird)
- signals and channels to look for in the metadata of the files and in the rig.json parameter file: Note that this have to exist in all of the sessions that will be processed
- 'sess' is unimportant here, but it comes handy if there is need to debug usin a single session

In [9]:
reload(et)
# for one example session

sess_par = {'bird': 's_b1238_22',
           'sess': '2022-02-12',
           'probes': ['probe_0'], #probes of interest
           'mic_list': ['microphone_0'], #list of mics of interest, by signal name in rig.json
           'stim_list': ['wav_syn'], # list of adc chans with the stimulus
           'nidq_ttl_list': ['wav_ttl'], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
           'sort': 2, #label for this sort instance
           }

exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], sort=sess_par['sort'])

ksort_folder = exp_struct['folders']['ksort']
raw_folder = exp_struct['folders']['sglx']

list all the epochs in a session, to check that it is finding what it has to find

In [10]:
sess_epochs = et.list_sgl_epochs(sess_par)
sess_epochs

2022-04-19 14:08:32,330 ceciestunepipe.file.bcistructure INFO     {'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/s_b1238_22', 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-12', 'sglx': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-12/sglx', 'kwik': '/scratch/earneodo/s_b1238_22/sglx/kwik/2022-02-12', 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1238_22/2022-02-12/sglx', 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx', 'tmp': '/scratch/earneodo/tmp', 'msort': '/scratch/earneodo/s_b1238_22/sglx/msort/2022-02-12', 'ksort': '/scratch/earneodo/s_b1238_22/sglx/ksort/2022-02-12/2', 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/2'}, 'files': {'par': '/scratch/earneodo/s_b1238_22/sglx/ksort/2022-02-12/2/params.json', 'set': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-12/sglx/settings.isf', 'rig': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-12/sglx/rig.json', 'kwd': '/scratch/earneodo/s_b123

['0716_g0', '1141_stim_g0', '1607_g0', '2212_g0']

#### define pre-processing steps for each epoch and for the session

###### for preprocessing one single epoch

In [11]:
reload(pre)
#one_epoch_dict = pre.preprocess_run(sess_par, exp_struct, sess_epochs[3])

<module 'ceciestunepipe.util.spikeextractors.preprocess' from '/mnt/cube/earneodo/repos/ceciestunepipe/ceciestunepipe/util/spikeextractors/preprocess.py'>

In [12]:
### sequentially process all runs of the sessions
def preprocess_session(sess_par: dict):
    logger.info('pre-process all runs of sess ' + sess_par['sess'])
    # get exp struct
    sess_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], sort=sess_par['sort'])
    # list the epochs
    sess_epochs = et.list_sgl_epochs(sess_par)
    logger.info('found epochs: {}'.format(sess_epochs))
    # preprocess all epochs
    epoch_dict_list = []
    for i_ep, epoch in enumerate(sess_epochs):
        try:
            exp_struct = et.sgl_struct(sess_par, epoch)
            one_epoch_dict = pre.preprocess_run(sess_par, exp_struct, epoch)
            epoch_dict_list.append(one_epoch_dict)
        except Exception as exc:
            warnings.warn('Error in epoch {}'.format(epoch), UserWarning)
            logger.info(traceback.format_exc)
            logger.info(exc)
            logger.info('Session {} epoch {} could not be preprocessed'.format(sess_par['sess'], epoch))
        
    return epoch_dict_list

#all_epoch_list = preprocess_session(sess_par)

## Process multiple sessions

In [13]:
sess_list = all_bird_sess
# fist implant, right hemisphere
#sess_list = ['2021-06-24', '2021-06-25', '2021-06-26', '2021-06-27', '2021-06-28', '2021-06-29', '2021-06-30']
sess_list = ['2022-01-17', '2022-01-18', '2022-01-19', '2022-01-20', # RH RA
             '2022-01-26', '2022-01-27','2022-01-28', '2022-01-29', '2022-01-30', #LH RA
            '2022-02-11', '2022-01-12', '2022-01-13'] #LH NCM

sess_list_ncm = ['2022-02-12']

In [14]:
sess_list = sess_list_ncm

In [15]:
sess_list

['2022-02-12']

In [16]:
all_sess_dict = {}

for one_sess in sess_list[:]:
    sess_par['sess'] = one_sess
    preprocess_session(sess_par)

2022-04-19 14:10:53,331 root         INFO     pre-process all runs of sess 2022-02-12
2022-04-19 14:10:53,331 root         INFO     pre-process all runs of sess 2022-02-12
2022-04-19 14:10:53,333 ceciestunepipe.file.bcistructure INFO     {'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/s_b1238_22', 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-12', 'sglx': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-12/sglx', 'kwik': '/scratch/earneodo/s_b1238_22/sglx/kwik/2022-02-12', 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1238_22/2022-02-12/sglx', 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx', 'tmp': '/scratch/earneodo/tmp', 'msort': '/scratch/earneodo/s_b1238_22/sglx/msort/2022-02-12', 'ksort': '/scratch/earneodo/s_b1238_22/sglx/ksort/2022-02-12/2', 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/2'}, 'files': {'par': '/scratch/earneodo/s_b1238_22/sglx/ksort/2022-02-12/2/params.json', 'set': '/mnt/sphere/

2022-04-19 14:11:36,051 ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor.spikeglxrecordingextractor INFO     getting ttl traces, chan [0 1 2 3 4 5 6 7]
2022-04-19 14:11:54,451 ceciestunepipe.util.wavutil INFO     sampling rate 40000
2022-04-19 14:11:54,451 ceciestunepipe.util.wavutil INFO     sampling rate 40000
2022-04-19 14:11:54,454 ceciestunepipe.util.wavutil INFO     saving (1, 632466585)-shaped array as wav in /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/0716_g0/wav_sync.wav
2022-04-19 14:11:54,454 ceciestunepipe.util.wavutil INFO     saving (1, 632466585)-shaped array as wav in /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/0716_g0/wav_sync.wav
2022-04-19 14:11:56,311 ceciestunepipe.util.wavutil INFO     saving (1, 632466585)-shaped array as npy in /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/0716_g0/wav_sync.npy
2022-04-19 14:11:56,311 ceciestunepipe.util.wavutil INFO     saving (1, 632466585)-shaped a

2022-04-19 14:13:03,810 ceciestunepipe.util.spikeextractors.preprocess INFO     getting syn patterns for ap_0
2022-04-19 14:13:03,827 ceciestunepipe.util.spikeextractors.preprocess INFO     File /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/0716_g0/ap_0_sync_dict.pkl not found or forced computation, getting the events
2022-04-19 14:13:03,827 ceciestunepipe.util.spikeextractors.preprocess INFO     File /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/0716_g0/ap_0_sync_dict.pkl not found or forced computation, getting the events
2022-04-19 14:13:03,829 ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor.spikeglxrecordingextractor INFO     getting ttl events, chan 0
2022-04-19 14:13:03,829 ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor.spikeglxrecordingextractor INFO     getting ttl events, chan 0
2022-04-19 14:13:03,830 ceciestunepipe.util.spikeextractors.extractors.readSGLX INFO     Extracting digital on ime

2022-04-19 14:34:38,524 ceciestunepipe.util.wavutil INFO     saving (1, 611852646)-shaped array as npy in /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/1141_stim_g0/wav_stim.npy
2022-04-19 14:34:38,524 ceciestunepipe.util.wavutil INFO     saving (1, 611852646)-shaped array as npy in /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/1141_stim_g0/wav_stim.npy
2022-04-19 14:34:50,373 ceciestunepipe.util.spikeextractors.preprocess INFO     Getting the onset/offset of stimuli from the 0 extracted analog channel
2022-04-19 14:34:50,373 ceciestunepipe.util.spikeextractors.preprocess INFO     Getting the onset/offset of stimuli from the 0 extracted analog channel
2022-04-19 14:34:51,532 ceciestunepipe.util.stimutil INFO     found 142.0 events
2022-04-19 14:34:51,532 ceciestunepipe.util.stimutil INFO     found 142.0 events
2022-04-19 14:34:51,571 ceciestunepipe.util.spikeextractors.preprocess INFO     saved onset/offset of trial events from the sine wave in /mnt/sp

2022-04-19 14:36:02,382 ceciestunepipe.util.spikeextractors.preprocess INFO     saving t_0 array to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/1141_stim_g0/lf_0_t0.npy
2022-04-19 14:36:02,382 ceciestunepipe.util.spikeextractors.preprocess INFO     saving t_0 array to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/1141_stim_g0/lf_0_t0.npy
2022-04-19 14:36:02,955 ceciestunepipe.util.spikeextractors.preprocess INFO     saving sync dict to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/1141_stim_g0/lf_0_sync_dict.pkl
2022-04-19 14:36:02,955 ceciestunepipe.util.spikeextractors.preprocess INFO     saving sync dict to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/1141_stim_g0/lf_0_sync_dict.pkl
2022-04-19 14:36:03,028 ceciestunepipe.util.spikeextractors.preprocess INFO     loading syn_dict from /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/1141_stim_g0/lf_0_sync_dict.pkl
2022-04-19 14:36:03,028 ceciestunepip

2022-04-19 14:57:29,333 ceciestunepipe.util.wavutil INFO     saving (1, 872783924)-shaped array as npy in /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/1607_g0/wav_mic.npy
2022-04-19 14:57:46,445 ceciestunepipe.util.spikeextractors.preprocess INFO     Getting stimulus channel(s) ['wav_syn']
2022-04-19 14:57:46,445 ceciestunepipe.util.spikeextractors.preprocess INFO     Getting stimulus channel(s) ['wav_syn']
2022-04-19 14:57:46,449 ceciestunepipe.util.wavutil INFO     sampling rate 40000
2022-04-19 14:57:46,449 ceciestunepipe.util.wavutil INFO     sampling rate 40000
2022-04-19 14:57:46,450 ceciestunepipe.util.wavutil INFO     saving (1, 872783924)-shaped array as wav in /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/1607_g0/wav_stim.wav
2022-04-19 14:57:46,450 ceciestunepipe.util.wavutil INFO     saving (1, 872783924)-shaped array as wav in /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/1607_g0/wav_stim.wav
2022-04-19 14:57:49,919 cecie

2022-04-19 14:58:54,315 ceciestunepipe.util.spikeextractors.extractors.readSGLX INFO     allocated array for syn channel of size (54550061,)
2022-04-19 14:58:54,315 ceciestunepipe.util.spikeextractors.extractors.readSGLX INFO     allocated array for syn channel of size (54550061,)
2022-04-19 15:00:07,057 ceciestunepipe.util.spikeextractors.extractors.readSGLX INFO     Threshold for logical hi is 0
2022-04-19 15:00:07,057 ceciestunepipe.util.spikeextractors.extractors.readSGLX INFO     Threshold for logical hi is 0
/mnt/cube/earneodo/repos/ceciestunepipe/ceciestunepipe/util/spikeextractors/extractors/spikeglxrecordingextractor/readSGLX.py:310: UserWarning: There were 3 errors in the recording
  warnings.warn('There were {} errors in the recording'.format(n_errors))
/mnt/cube/earneodo/repos/ceciestunepipe/ceciestunepipe/util/spikeextractors/extractors/spikeglxrecordingextractor/readSGLX.py:311: UserWarning: You probably want to check those
  warnings.warn('You probably want to check thos

2022-04-19 15:33:23,507 ceciestunepipe.util.spikeextractors.preprocess INFO     PREPROCESSING sess 2022-02-12 | epoch 2212_g0
2022-04-19 15:33:23,507 ceciestunepipe.util.spikeextractors.preprocess INFO     PREPROCESSING sess 2022-02-12 | epoch 2212_g0
2022-04-19 15:33:23,509 ceciestunepipe.util.spikeextractors.preprocess INFO     getting extractors
2022-04-19 15:33:23,509 ceciestunepipe.util.spikeextractors.preprocess INFO     getting extractors
2022-04-19 15:33:23,945 ceciestunepipe.util.spikeextractors.preprocess INFO     Got sglx recordings for keys ['nidq', 'lf_0', 'ap_0']
2022-04-19 15:33:23,945 ceciestunepipe.util.spikeextractors.preprocess INFO     Got sglx recordings for keys ['nidq', 'lf_0', 'ap_0']
2022-04-19 15:33:23,947 ceciestunepipe.util.spikeextractors.preprocess INFO     Getting microphone channel(s) ['microphone_0']
2022-04-19 15:33:23,947 ceciestunepipe.util.spikeextractors.preprocess INFO     Getting microphone channel(s) ['microphone_0']
2022-04-19 15:33:23,948 ceci

2022-04-19 15:36:21,876 ceciestunepipe.util.spikeextractors.preprocess INFO     Getting sync signals for imec streams: ['lf_0', 'ap_0']
2022-04-19 15:36:21,876 ceciestunepipe.util.spikeextractors.preprocess INFO     Getting sync signals for imec streams: ['lf_0', 'ap_0']
2022-04-19 15:36:21,877 ceciestunepipe.util.spikeextractors.preprocess INFO     getting syn patterns for lf_0
2022-04-19 15:36:21,877 ceciestunepipe.util.spikeextractors.preprocess INFO     getting syn patterns for lf_0
2022-04-19 15:36:21,910 ceciestunepipe.util.spikeextractors.preprocess INFO     File /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/2212_g0/lf_0_sync_dict.pkl not found or forced computation, getting the events
2022-04-19 15:36:21,910 ceciestunepipe.util.spikeextractors.preprocess INFO     File /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/2212_g0/lf_0_sync_dict.pkl not found or forced computation, getting the events
2022-04-19 15:36:21,911 ceciestunepipe.util.spikeextra

2022-04-19 15:58:51,207 ceciestunepipe.util.spikeextractors.preprocess INFO     getting syn patterns for nidq
2022-04-19 15:58:51,220 ceciestunepipe.util.spikeextractors.preprocess INFO     loading syn_dict from /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/2212_g0/nidq_sync_dict.pkl
2022-04-19 15:58:51,220 ceciestunepipe.util.spikeextractors.preprocess INFO     loading syn_dict from /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/2212_g0/nidq_sync_dict.pkl
2022-04-19 15:58:51,372 ceciestunepipe.util.spikeextractors.preprocess INFO     getting syn patterns for wav
2022-04-19 15:58:51,372 ceciestunepipe.util.spikeextractors.preprocess INFO     getting syn patterns for wav
2022-04-19 15:58:51,400 ceciestunepipe.util.spikeextractors.preprocess INFO     loading syn_dict from /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/2212_g0/wav_sync_dict.pkl
2022-04-19 15:58:51,400 ceciestunepipe.util.spikeextractors.preprocess INFO     loading syn_dict 

In [17]:
sess_par

{'bird': 's_b1238_22',
 'sess': '2022-02-12',
 'probes': ['probe_0'],
 'mic_list': ['microphone_0'],
 'stim_list': ['wav_syn'],
 'nidq_ttl_list': ['wav_ttl'],
 'sort': 2}

## search bouts for those sessions

In [18]:
from ceciestunepipe.util.sound import boutsearch as bs
from ceciestunepipe.util import wavutil as wu

from joblib import Parallel, delayed
import pickle
import sys

In [13]:
hparams = { # default parameters work well for starling
    # spectrogram
    'num_freq':1024, #1024# how many channels to use in a spectrogram #
    'preemphasis':0.97, 
    'frame_shift_ms':5, # step size for fft
    'frame_length_ms':10, #128 # frame length for fft FRAME SAMPLES < NUM_FREQ!!!
    'min_level_db':-55, # minimum threshold db for computing spe 
    'ref_level_db':110, # reference db for computing spec
    'sample_rate':None, # sample rate of your data
    
    # spectrograms
    'mel_filter': False, # should a mel filter be used?
    'num_mels':1024, # how many channels to use in the mel-spectrogram
    'fmin': 500, # low frequency cutoff for mel filter
    'fmax': 12000, # high frequency cutoff for mel filter
    
    # spectrogram inversion
    'max_iters':200,
    'griffin_lim_iters':20,
    'power':1.5,

    # Added for the searching
    'read_wav_fun': bs.read_wav_chan, # function for loading the wav_like_stream (has to returns fs, ndarray)
    'file_order_fun': bs.sess_file_id, # function for extracting the file id within the session
    'min_segment': 30, # Minimum length of supra_threshold to consider a 'syllable' (ms)
    'min_silence': 3000, # Minmum distance between groups of syllables to consider separate bouts (ms)
    'min_bout': 3000, # min bout duration (ms)
    'peak_thresh_rms': 0.55, # threshold (rms) for peak acceptance,
    'thresh_rms': 0.25, # threshold for detection of syllables
    'mean_syl_rms_thresh': 0.3, #threshold for acceptance of mean rms across the syllable (relative to rms of the file)
    'max_bout': 180000, #exclude bouts too long
    'l_p_r_thresh': 100, # threshold for n of len_ms/peaks (typycally about 2-3 syllable spans
    
    'waveform_edges': 1000, #get number of ms before and after the edges of the bout for the waveform sample
    
    'bout_auto_file': 'bout_auto.pickle', # extension for saving the auto found files
    'bout_curated_file': 'bout_checked.pickle', #extension for manually curated files (coming soon)
    }

In [14]:
all_sessions = sess_list[:]
#all_sessions = ['2021-06-15']

for sess in all_sessions:
    sess_par['sess'] = sess
    sess_bout_pd = sb.get_all_day_bouts(sess_par, hparams, n_jobs=1, ephys_software='sglx')
    sb.save_auto_bouts(sess_bout_pd, sess_par, hparams, software='sglx')
    #sess_bouts_folder = os.path.join(exp_struct['folders']['derived'], 'bouts')
    #bouts_to_wavs(sess_bout_pd, sess_par, hparams, sess_bouts_folder)

2022-04-12 08:45:36,275 ceciestunepipe.pipeline.searchbout INFO     Will search for bouts through all session s_b1238_22, 2022-02-12
2022-04-12 08:45:36,286 ceciestunepipe.pipeline.searchbout INFO     getting wav files from/mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx
2022-04-12 08:45:36,287 ceciestunepipe.pipeline.searchbout INFO     Found 4 files
2022-04-12 08:45:36,288 ceciestunepipe.util.sound.boutsearch INFO     Getting bouts for long file /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/0716_g0/wav_mic.wav
2022-04-12 08:45:36,312 ceciestunepipe.util.sound.boutsearch INFO     splitting file into 8 chunks


  0%|          | 0/8 [00:00<?, ?it/s]

2022-04-12 08:50:11,820 ceciestunepipe.util.sound.boutsearch INFO     Getting bouts for long file /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/1141_stim_g0/wav_mic.wav
2022-04-12 08:50:11,823 ceciestunepipe.util.sound.boutsearch INFO     splitting file into 7 chunks


  0%|          | 0/7 [00:00<?, ?it/s]

2022-04-12 08:54:36,112 ceciestunepipe.util.sound.boutsearch INFO     Getting bouts for long file /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/1607_g0/wav_mic.wav
2022-04-12 08:54:36,116 ceciestunepipe.util.sound.boutsearch INFO     splitting file into 10 chunks


  0%|          | 0/10 [00:00<?, ?it/s]

2022-04-12 09:00:19,138 ceciestunepipe.util.sound.boutsearch INFO     Getting bouts for long file /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/2212_g0/wav_mic.wav
2022-04-12 09:00:19,152 ceciestunepipe.pipeline.searchbout INFO     Error in file /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/sglx/2212_g0/wav_mic.wav
2022-04-12 09:00:19,158 ceciestunepipe.pipeline.searchbout INFO     Traceback (most recent call last):
  File "/mnt/cube/earneodo/repos/ceciestunepipe/ceciestunepipe/pipeline/searchbout.py", line 77, in get_file_bouts
    bpd, _ = bs.get_bouts_in_long_file(i_path, hparams)
  File "/mnt/cube/earneodo/repos/ceciestunepipe/ceciestunepipe/util/sound/boutsearch.py", line 279, in get_bouts_in_long_file
    s_f, wav_i = hparams['read_wav_fun'](file_path)
  File "/mnt/cube/earneodo/repos/ceciestunepipe/ceciestunepipe/util/sound/boutsearch.py", line 26, in read_wav_chan
    return s_f, x[:, chan_id]
  File "/home/earneodo/anaconda3/envs/unepipe/lib/python

TypeError: save_auto_bouts() got an unexpected keyword argument 'ephys_software'

In [17]:
sess_bout_pd.head(2)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,confusing,valid_waveform,valid,spectrogram
0,15670,23865,626800,954600,"[12.426594779175346, 15.346236536485952, 55.49...",11.019281,241.030983,False,/mnt/sphere/speech_bci/derived_data/s_b1238_22...,8195,"[[15670, 15700], [15820, 15910], [16340, 16395...",11,"[2, 13, 27, 39, 75, 122, 136, 147, 161, 170, 1...",46,178.152174,"[74, 147, 145, 99, 112, 88, 56, 98, 126, 142, ...",True,True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,32475,38055,1299000,1522200,"[10.02672970768318, 15.310227594990456, 15.155...",11.019281,168.335634,False,/mnt/sphere/speech_bci/derived_data/s_b1238_22...,5580,"[[32475, 32570], [32590, 32680], [34200, 34245...",9,"[7, 14, 26, 33, 126, 316, 347, 363, 383, 513, ...",29,192.413793,"[21, 22, 20, 1, 18, 20, 4, 0, -24, -14, -6, -2...",True,True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [18]:
sb.save_auto_bouts(sess_bout_pd, sess_par, hparams, software='sglx')

2022-04-12 09:17:02,653 ceciestunepipe.pipeline.searchbout INFO     saving bouts pandas to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/bouts_sglx/bout_auto.pickle
2022-04-12 09:17:31,269 ceciestunepipe.pipeline.searchbout INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/bouts_sglx/bout_search_params.pickle
2022-04-12 09:17:31,270 ceciestunepipe.util.sound.boutsearch INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-12/bouts_sglx/bout_search_params.pickle


# debug

## debug search_bout

In [21]:
## look for a single file
sess = sess_list[0]
exp_struct = et.get_exp_struct(sess_par['bird'], sess, ephys_software='sglx')
source_folder = exp_struct['folders']['derived']
wav_path_list = et.get_sgl_files_epochs(source_folder, file_filter='*wav_mic.wav')
wav_path_list.sort()
logger.info('Found {} files'.format(len(wav_path_list)))
print(wav_path_list)

2021-09-22 15:13:39,371 root         INFO     Found 4 files


['/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/0712_g0/wav_mic.wav', '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/1255_g0/wav_mic.wav', '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/1740_g0/wav_mic.wav', '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/2118_g0/wav_mic.wav']


In [22]:
one_file = wav_path_list[0]

In [ ]:
reload(bs)
epoch_bout_pd, epoch_wav = bs.get_bouts_in_long_file(wav_path_list[0], hparams)

2021-09-22 15:13:45,924 ceciestunepipe.util.sound.boutsearch INFO     Getting bouts for long file /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/0712_g0/wav_mic.wav


tu vieja file /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/0712_g0/wav_mic.wav


2021-09-22 15:13:45,962 ceciestunepipe.util.sound.boutsearch INFO     splitting file into 5 chunks


  0%|          | 0/5 [00:00<?, ?it/s]